In [13]:
from google.colab import drive
drive.mount('/content/gdrive')

!pip install ultralytics

In [21]:
from ultralytics import YOLO
import json
import glob
import os
from pathlib import Path

In [14]:
directory_name = "gdrive/MyDrive/Colab Notebooks/GESTALT/"
json_filename = "gdrive/MyDrive/Colab Notebooks/GESTALT/metadata.json"

In [51]:
def detect_tags_from_jpgs_in_directory(directory_name, json_filename):
    # Find all jpegs in the directory
    list_of_img_files = glob.glob(os.path.join(directory_name,"*.jpg"))

    # Read in json file
    json_dict = json.load(open(json_filename))

    # Instantiate model
    model = YOLO("yolov8m.pt")

    # Run images through model and get results
    results = model.predict(list_of_img_files, verbose=False)

    # Check that we got results for every image
    assert len(results) == len(list_of_img_files)

    # Loop over each image
    for idx in range(len(list_of_img_files)):
        file_id = Path(list_of_img_files[idx]).stem  # extract filename without path or extension
        result = results[idx]
        tags = []
        coords = []
        probs = []
        # Loop over each object detected in the image
        for box in result.boxes:
            class_id = result.names[box.cls[0].item()]
            cords = box.xyxy[0].tolist()
            cords = [round(x) for x in cords]
            conf = round(box.conf[0].item(), 2)

            tags.append(class_id)  # object type like person or bench
            coords.append(cords)  # coordinates in the image like [121, 632, 207, 732]
            probs.append(conf)  # confidence score like 0.81

        json_dict[file_id].update({"objects":tags, "coordinates":coords, "probabilities":probs})

    return json.dumps(json_dict)

In [52]:
output = detect_tags_from_jpgs_in_directory(directory_name, json_filename)